# 量子谱层析

*版权所有 (c) 2022 百度量子计算研究所，保留所有权利。*

## 内容概要
在迈向大规模量子计算的道路上，一个核心挑战是提高量子门保真度。那么如何衡量一个量子门的保真度也成为了一个值得思考的问题。
量子谱层析 (Spectral Quantum Tomography, SQT) 可以有效地提取含噪的量子门的谱信息，并且能够抵抗量子态制备过程和测量（State Preparation and Measurement, SPAM）中的噪声。该方法最大的优点就是简便高效，只需要重复地利用目标量子门 $\mathcal{N}$，这一点远远优于随机基准测量、量子门层析和增强层析等方法。另外一个优点是该方法可以提取不具备马尔可夫性质量子门的信息。

本教程将介绍谱层析的理论及其在[百度量子平台](https://quantum.baidu.com/)上的实践。


## 理论

### Pauli Transfer Matrix 表达
一个含噪量子门 $\mathcal{N}$ 的谱信息，可以通过求解它 PTM 表达下矩阵 $[\mathcal{N}]$ 的特征值获取。我们知道，一个酉门可以被认为是一个半正定和保迹 (TPCP) 的线性映射。对 $n$ 量子比特的希尔伯特空间而言，我们用一组标准化的 Pauli 矩阵 $\frac{1}{\sqrt{2}}\left\{P_i\right\}$ ($i = 0, \dots, N$ 其中 $N+1 = 4^n$) 组成该空间的一组基。在 PTM 表达下，我们可以通过以下式子来定义一个 TPCP 线性映射对应的 PTM 每个元素：

$$
    [\mathcal{N}]_{ij} = \textrm{Tr}\left[ P_i \mathcal{N}(P_{j})\right],\; i, j = 0, \dots, N.
$$

我们可以将 PTM 表达重写为以下形式：

$$
    [\mathcal{N}] = \begin{pmatrix}
        1 & 0 \\ \mathbf{n} & T^\mathcal{N}
    \end{pmatrix}.
$$

这里的 $T^\mathcal{N}$ 是一个 $N \times N$ 的实矩阵（因为是在 Pauli 基下展开的）， $\mathbf{n}$ 是一个 $N$ 维度的列向量。
由于 $\mathcal{N}$ 是一个保迹的线性映射，它对应 PTM 的第一行首元素为 $1$ 其余元素全为 $0$ 。对应于一个酉门（$\mathcal{N}(I) = I$），那么则有 $\mathbf{n} = \mathbf{0}$ 。


### 谱层析方法
我们将态制备过程中的噪声建模为一个未知的 TPCP 线性映射 $\mathcal{N}_{prep}$，同样地我们将测量过程中的噪声建模为 $\mathcal{N}_{meas}$。然后我们应用 $k$ 次的目标量子门 $\mathcal{N}$，对应于$\mathcal{N}^k$。我们设定 $k$ 的最大值为 $K$，对于 $k = 0,1,\dots,K$ 我们定义以下序列：

$$
    g(k) = \sum_i \textrm{Tr} \left[ P_i \mathcal{N}_{meas} \circ \mathcal{N}^k \circ \mathcal{N}_{prep}(P_i)\right].
$$

获取序列 $g(k)$ 的实验过程如下：
1. 选取一个 Pauli 矩阵 $P_i$，满足 $\textrm{Tr} \left[P_i\right] = 0$ 条件（也就是不选择 $I$ 作为输入）。
2. 根据 (1) 过程中选择的 $P_i$ 构造态制备电路。（注意：态制备这一过程存在噪声，对应于上式的 $\mathcal{N}_{prep}$）
3. 将目标量子门作用 $k$ 次，并在后面铺上对应于 $P_i$ 的测量电路。（注意：该过程分别对应于 $\mathcal{N}^k$ 和 $\mathcal{N}_{meas}$）
4. 更换 $k$ 的取值和不同的 $P_i$，重复 $(1-3)$ 步骤，然后处理实验数据。

通过以上步骤我们最终会得到一个序列 $g(k)$ ，下面将介绍如何根据这一序列获取矩阵 $T$ 的特征值。
我们先来看没有态制备和测量（SPAM）误差情况下如何处理这一序列，也就是说 $\mathcal{N}_{meas}$ 和 $\mathcal{N}_{prep}$ 对应于一个单位矩阵，我们则有下式：

$$
    g^{ideal}(k) = \sum_{i} \textrm{Tr} \left[ P_i \mathcal{N}^k(P_i)\right] = \sum_i (T^k)_{ii}
    = \textrm{Tr} \left[T^k\right] = \sum_{i=1}^{N} \lambda_i,
$$

这里的 $\left\{ \lambda_i \right\}$ 都是矩阵 $T$ 的特征值（提示：最后一个等式需要 $T$ 满足相似对角化条件）。
当 SPAM 误差不可被忽略时，我们则有：

$$
\begin{aligned}
    g(k) &= \textrm{Tr} \left[T_{meas} T^k T_{prep}\right] = \textrm{Tr} \left[T_{meas} VD^kV^{-1} T_{prep}\right] \\
    &= \textrm{Tr} \left[V^{-1} T_{prep} T_{meas} V D^k\right] \\
    &= \textrm{Tr} \left[A_{SPAM} D^k\right] \\
    &= \sum_i A_{i} \lambda_i^k.
\end{aligned}
$$

这里的 $T_{meas}$ and $T_{prep}$ 分别对应于 $\mathcal{N}_{meas}$ 和 $\mathcal{N}_{prep}$ PTM 表达下 $T$ 子矩阵。

### Matrix-pencil 方法
Matrix-pencil 方法是一种经典的信号处理方法，可用于从含噪信号 $g(k)=\sum_i A_{i}\lambda_i^k$ 中获取特征值 $\lambda_i$ 和 振幅系数 $A_{i}$ 。为了获取准确的特征值信息，该方法要求满足 $K \geq 2N-2$ 条件，其中 $N = 4^n - 1$ 。然后我们介绍一个重要的参数，被称为 pencil 参数 $L$，它决定了矩阵 $Y$ 的形状。 $Y$ 矩阵的形状为 $(K-L+1) \times (L+1)$ 的矩阵，它的具体形式为：

$$
\begin{align}
    Y = \begin{pmatrix}
        g(0) & g(1) & \cdots & g(L) \\
        g(1) & g(2) & \cdots & g(L+1) \\
        g(2) & & & \vdots \\
        \vdots & & \ddots & \vdots \\
        g(K-L) & \cdots & \cdots & g(K)
    \end{pmatrix} =
    \sum_{i=1}^{N} A_i \begin{pmatrix}
        1 & \lambda_i & \cdots & \lambda_i^L \\
        \lambda_i & \lambda_i^2 & \cdots & \lambda_i^{L+1} \\
        \lambda_i^2 & & & \vdots \\
        \vdots & & \ddots & \vdots \\
        \lambda_i^{K-L} & \cdots & \cdots & \lambda_i^{K}
    \end{pmatrix}.
\end{align}
$$

然后我们再介绍两个 $Y$ 矩阵的子矩阵：

$$
\begin{align}
    G_0 = \begin{pmatrix}
        g(0) & g(1) & \cdots & g(L-1) \\
        g(1) & g(2) & \cdots & g(L) \\
        g(2) & & & \vdots \\
        \vdots & & \ddots & \vdots \\
        g(K-L) & \cdots & \cdots & g(K-1)
    \end{pmatrix},\;
    G_1 = \begin{pmatrix}
        g(1) & g(2) & \cdots & g(L) \\
        g(2) & g(3) & \cdots & g(L+1) \\
        g(3) & & & \vdots \\
        \vdots & & \ddots & \vdots \\
        g(K-L+1) & \cdots & \cdots & g(K)
    \end{pmatrix}.
\end{align}
$$

下面将分有无噪声两种情况，分别求解序列对应的特征值。

#### 处理无噪声信号

通过 SVD ，我们可以将 $G_0$ 和 $G_1$ 改写为：

$$
\begin{align}
    G_1 &= \Lambda_1 R \Lambda_0 \Lambda_2, \\
    G_0 &= \Lambda_1 R \Lambda_2.
\end{align}
$$

我们不难发现以下关系：

$$
\begin{align}
    G_1 - \lambda G_0 = \Lambda_1 R \left(\Lambda_0 - \lambda I\right) \Lambda_2,
\end{align}
$$

等效地，序列特征值求解问题转换成经典矩阵特征值求解问题，

$$
\begin{align}
    G_0^+ G_1 - \lambda I = 0,
\end{align}
$$

这里的 $G_0^+$ 是 $G_0$ 的广义逆矩阵，它的具体表达式如下：

$$
\begin{align}
    G_0^+ = \left(G_0^\dagger G_0\right)^{-1} G_0^\dagger.
\end{align}
$$

#### 处理含噪声信号

对矩阵 $Y$ 进行 SVD 处理：

$$
\begin{align}
    Y = U \Sigma V^\dagger.
\end{align}
$$

这里的 $U$ 和 $V$ 都是酉矩阵，由 $YY^\dagger$ 和 $Y^\dagger Y$ 的特征值组成；$\Sigma$ 是一个包含矩阵 $Y$ 奇异值的对角矩阵。
然后我们考虑一个『过滤』矩阵，$V^{'}$, 仅包含 $V$的 $N$ 个最大奇异值对应的列向量：

$$
\begin{align}
    V = \left[v_1, v_2, \dots, v_N\right],
\end{align}
$$

因此我们有：

$$
\begin{align}
    G_0 &= U \Sigma^{'} V_0^{'\dagger}, \\
    G_1 &= U \Sigma^{'} V_1^{'\dagger},
\end{align}
$$

这里我们通过相似的『过滤』操作得到 $V_0^{'\dagger}$ 、$V_1^{'\dagger}$ 以及 $\Sigma^{'}$。

$$
\begin{align}
    G_1 - \lambda G_0 \Rightarrow G_0 G_1^+ - \lambda I.
\end{align}
$$

上式中对应的特征值等效于求解如下矩阵的特征值：

$$
\begin{align}
    V_1^{'\dagger} - \lambda V_0^{'\dagger} \Rightarrow V_0^{'\dagger} \left(V_1^{'\dagger}\right)^+ - \lambda I.
\end{align}
$$

## 实践

### 单量子比特门
首先，我们需要导入一些必要的包。

In [7]:
import numpy as np
import QCompute
import qcompute_qep.tomography as tomography
import qcompute_qep.quantum.pauli as pauli
import qcompute_qep.utils.circuit

然后，我们需要搭建一个理想的 $R_x(\frac{1}{4} \pi)$ 门，并计算理想情况下它 PTM 表达下的特征值。

In [ ]:
qp = QCompute.QEnv()  # qp is short for "quantum program", instance of QProgram
qp.Q.createList(1)

QCompute.RX(np.math.pi / 4)(qp.Q[0])

# Compute numerically the ideal R_x for reference
ideal_rx = qcompute_qep.utils.circuit.circuit_to_unitary(qp)
ideal_ptm = pauli.unitary_to_ptm(ideal_rx).data
print("the ideal Rotation x is \n", ideal_rx)
print("the ideal PTM of rotation x is \n", ideal_ptm)

# calculate the eigenvalues of PTM representation
ideal_eigenvalues, _ = np.linalg.eig(ideal_ptm[1:, 1:])
print("the ideal eigenvalues of PTM representation is \n", ideal_eigenvalues)


接下来我们需要选择量子计算机的类型，可以选择模拟器或量子计算机真机，这里我们选择的是百度本地量子模拟器。剩下的步骤就很简单了，只需要调用我们的 SpectralTomography 模块则可完成特征值的提取。

In [ ]:
# Please login the "Quantum Leaf" platform (https://quantum-hub.baidu.com/) to get Token
QCompute.Define.hubToken = "Token"

qc = QCompute.BackendName.CloudBaiduQPUQian

st = tomography.SpectralTomography()
noisy_eigvals = st.fit(qp, qc, k=50, l=30)

print("the eigenvalues we estimate is ", noisy_eigvals)

我们可以利用 matplotlib 来进行数据可视化。

In [ ]:
import matplotlib.pyplot as plt
from cmath import *

ax = plt.subplot(polar=True)

ax.set_rlim(0.99, 1.01)
noisy_data = np.zeros((2, np.size(noisy_eigvals)), dtype=float)
ideal_data = np.zeros((2, np.size(ideal_eigenvalues)), dtype=float)
for i, val in enumerate(noisy_eigvals):
    noisy_data[:, i] = np.asarray(polar(val))

for i, val in enumerate(ideal_eigenvalues):
    ideal_data[:, i] = np.asarray(polar(val))

ax.scatter(noisy_data[1, :], noisy_data[0, :], c='blue', label='noisy')
ax.scatter(ideal_data[1, :], ideal_data[0, :], c='red', label='ideal')
plt.legend()
plt.show()


![SQT](./figures/sqt-rx-output.png "图1：我们对单比特测试结果进行可视化。")


### 双量子比特门

同样地，我们首先创建一个包含 CNOT 门的量子程序，并计算它的理想特征值。

In [ ]:
qp = QCompute.QEnv()  # qp is short for "quantum program", instance of QProgram
qp.Q.createList(2)

# Manually decompose the CNOT gate using the CZ gate, where CNOT: q1 -> q0
QCompute.H(qp.Q[0])
QCompute.CZ(qp.Q[1], qp.Q[0])
QCompute.H(qp.Q[0])

# Compute numerically the ideal CNOT for reference
ideal_cnot = qcompute_qep.utils.circuit.circuit_to_unitary(qp)
ideal_ptm = pauli.unitary_to_ptm(ideal_cnot).data
print("the ideal CNOT is \n", ideal_cnot)
print("the ideal PTM of CNOT is \n", ideal_ptm[1:, 1:])

# calculate the eigenvalues of PTM representation
ideal_eigenvalues, _ = np.linalg.eig(ideal_ptm)
print("the ideal eigenvalues of PTM representation is \n", ideal_eigenvalues)

qc = QCompute.BackendName.CloudBaiduQPUQian


我们都知道 CNOT 门对应的 $T^\mathcal{N}$ 包含 $15$ 个特征值，但这里包含很多数值相同的特征值（实际特征值为 $\pm 1$ ）。我们可以通过改变参数 $N$ 来让我们的实验结果更加逼近理论值，这里我们选择的是 $N=2$。

In [ ]:
# Initialize a ProcessTomography instance
st = tomography.SpectralTomography()
# Call the tomography procedure and obtain the noisy CNOT gate
noisy_eigvals = st.fit(qp, qc, k=50, l=30, N=2)

我们可以利用 matplotlib 来进行数据可视化。

In [ ]:
import matplotlib.pyplot as plt
from cmath import *

ax = plt.subplot(polar=True)

ax.set_rlim(0.99, 1.01)
noisy_data = np.zeros((2, np.size(noisy_eigvals)), dtype=float)
ideal_data = np.zeros((2, np.size(ideal_eigenvalues)), dtype=float)

for i, val in enumerate(noisy_eigvals):
    noisy_data[:, i] = np.asarray(polar(val))

for i, val in enumerate(ideal_eigenvalues):
    ideal_data[:, i] = np.asarray(polar(val))

# print("ideal data:\n", ideal_data)
# print("noisy data:\n", noisy_data)

ax.scatter(noisy_data[1, :], noisy_data[0, :], c='blue', label='noisy')
ax.scatter(ideal_data[1, :], ideal_data[0, :], c='red', label='ideal')
plt.legend()
plt.show()



![SQT example](./figures/sqt-cnot-output.png "图2：我们对双比特测试结果进行可视化。")


## 总结
本教程旨在介绍 SQT 的原理以及如何在百度量子计算平台上实现。

## 参考资料
[1] Helsen, Jonas, Francesco Battistel, and Barbara M. Terhal. "Spectral quantum tomography." [npj Quantum Information](https://www.nature.com/articles/s41534-019-0189-0) 5.1 (2019): 1-11.

[2] Sarkar, Tapan K., and Odilon Pereira. "Using the matrix pencil method to estimate the parameters of a sum of complex exponentials." [IEEE Antennas and Propagation Magazine](https://ieeexplore.ieee.org/abstract/document/370583/) 37.1 (1995): 48-55.